In [39]:
import requests
import pandas as pd
import time

In [21]:
key = '72e4097c974d4ddebaa4a2a2674d70d3'
endpoint = "https://api.cognitive.microsoft.com/bing/v7.0/search"
headers = {"Ocp-Apim-Subscription-Key": key}
params = {"q": '', "textDecorations": False, "textFormat": "HTML", 'count': 1,'responseFilter':'Webpages',}

In [22]:
# MA_public_df = pd.read_csv('MA_public_schools.csv')
# MA_private_df = pd.read_csv('MA_private_schools.csv')

In [23]:
# test_schools_20 = pd.concat([MA_public_df.sample(n=10),MA_private_df.sample(n=10)]).astype('object')
# test_schools_20 = test_schools_20.reset_index().drop('index',axis = 1)
# test_schools_20.to_csv("MA_test_schools_20.csv")

In [24]:
test_schools_20 = pd.read_csv('MA_test_schools_20.csv', index_col = 0)

In [53]:
#This currently has an 85% success rate
def name_and_school(row):
    search_term = row['Org Name']
    if ':' in search_term:
        search_term = search_term.split(':')[1]
    if 'school' not in search_term.lower():
        search_term += ' School'
    return search_term

In [54]:
def get_url(row,searchTermFunc):
    params['q'] = searchTermFunc(row)
    response = requests.get(endpoint, headers=headers, params=params)
    search_results = response.json()
    url = search_results['webPages']['value'][0]['url']
    #To prevent calling the API too many times per second
    time.sleep(.5)
    return url

In [55]:
#Be careful running this as it performs as many API calls as the test_schools_df and that is limited
test_schools_20['url'] = test_schools_20.apply(get_url,args = [name_and_school],axis=1)

In [56]:
test_schools_20.head()

,Org Name,Org Type,Address 1,Town,State,Zip,url
0,Acushnet: Acushnet Elementary School,Public School,800 Middle Road,Acushnet,MA,2743,https://www.acushnetschools.us/elementary-scho...
1,Franklin: Davis Thayer,Public School,137 West Central Street,Franklin,MA,2038,https://elementaryschools.org/directory/ma/cit...
2,Revere: Revere High,Public School,101 School Street,Revere,MA,2151,https://www.revereschools.org/
3,Pentucket: Helen R Donaghue School,Public School,24 Union Street Ext.,Merrimac,MA,1860,http://hrd-pentucketrsd-ma.schoolloop.com/
4,Quincy: Clifford H Marshall Elementary,Public School,200 Moody St Ext,Quincy,MA,2169,https://marshall.quincypublicschools.com/


In [57]:
for i,row in test_schools_20.iterrows():
    print(row['Org Name'],row['url'])

Acushnet: Acushnet Elementary School https://www.acushnetschools.us/elementary-school/resources
Franklin: Davis Thayer https://elementaryschools.org/directory/ma/cities/franklin/davis-thayer/250501000736/
Revere: Revere High https://www.revereschools.org/
Pentucket: Helen R Donaghue School http://hrd-pentucketrsd-ma.schoolloop.com/
Quincy: Clifford H Marshall Elementary https://marshall.quincypublicschools.com/
Swansea: Elizabeth S Brown https://www.swanseaschools.org/
Newton: Lincoln-Eliot https://www.newton.k12.ma.us/Page/25
Hopedale: Memorial https://www.memorial-elementary-school.com/
Boston: Lyon Upper 9-12 https://www.usnews.com/education/best-high-schools/massachusetts/districts/boston-public-schools/lyon-upper-9-12-91967
Lynnfield: Lynnfield High https://www.lynnfield.k12.ma.us/
Alhuda Academy http://alhudaacademy.org/
Saint Augustine https://staugustinesunioncity.com/
MetroWest Jewish Day School https://www.mwjds.org/
New Covenant School Inc. https://www.facebook.com/newcovena